# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

%matplotlib inline

In [ ]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("newsapi")

In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [ ]:
# Fetch the Bitcoin news articles
btc_articles = newsapi.get_everything(q="Bitcoin", language="en", page_size = 100, sort_by = "relevancy")

In [ ]:
# Fetch the Ethereum news articles
eth_articles = newsapi.get_everything(q="Ethereum", language="en",page_size = 100, sort_by = "relevancy")

In [ ]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiments = []

for article in btc_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
btc_df = pd.DataFrame(btc_sentiments)
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

In [ ]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiments = []

for article in eth_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    

eth_df = pd.DataFrame(eth_sentiments)
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
btc_df.describe()

In [ ]:
# Describe the Ethereum Sentiment
eth_df.describe()

In [ ]:
print(btc_df['compound'].max())
print(eth_df['compound'].max())
print(btc_df['positive'].max())
print(eth_df['positive'].max())

### Questions:

Q: Which coin had the highest mean positive score?

A: BTC

Q: Which coin had the highest compound score?

A: ETH

Q. Which coin had the highest positive score?

A: ETH

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = stopwords.words('english')


# Expand the default stopwords list if necessary
#sw_addon = 


In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Remove the stop words
    sw= stopwords.words('english')
    
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('',text)
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize words into root words
    lem_words = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    tokens = [word.lower() for word in lem_words if word.lower() not in sw]
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
tokens = []
for text in btc_df['text']:
    tokens.append(tokenizer(text))

btc_df['tokens'] = tokens
btc_df.head()

In [ ]:
# Create a new tokens column for Ethereum
tokens = []
for text in eth_df['text']:
    tokens.append(tokenizer(text))

eth_df['tokens'] = tokens
eth_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
btc_string = ''.join(btc_df['text'])
btc_grams = ngrams(tokenizer(btc_string), n=2)

In [ ]:
# Generate the Ethereum N-grams where N=2
eth_string = ''.join(eth_df['text'])
eth_grams = ngrams(tokenizer(eth_string), n=2)

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
token_count(tokenizer(btc_string), 10)

In [ ]:
# Use token_count to get the top 10 words for Ethereum
token_count(tokenizer(eth_string), 10)

In [ ]:
# Top 10 pairing of words for Bitcoin with N=2
token_count(btc_grams, 10)

In [ ]:
# Top 10 pairing of words for Ethereum with N=2
token_count(eth_grams, 10)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
wc = WordCloud().generate(btc_string)
plt.title('Bitcoin WordCloud', fontsize = 40)
plt.imshow(wc)

In [ ]:
# Generate the Ethereum word cloud
wc = WordCloud().generate(eth_string)
plt.title('Ethereum WordCloud', fontsize = 40)
plt.imshow(wc)

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
btc_string = ''.join(btc_df['text'])

In [ ]:
# Run the NER processor on all of the text
btc_doc = nlp(btc_string)

# Add a title to the document
btc_doc.user_data["title"] = "Bitcoin NER"

In [ ]:
# Render the visualization
displacy.render(btc_doc, style='ent')

In [ ]:
# List all Entities
[ent.text for ent in btc_doc.ents][:10]
#Only showing 10 otherwise will be very long

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
eth_string = ''.join(eth_df['text'])

In [ ]:
# Run the NER processor on all of the text
eth_doc = nlp(eth_string)

# Add a title to the document
eth_doc.user_data["title"] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(eth_doc, style='ent')

In [ ]:
# List all Entities
[ent.text for ent in eth_doc.ents][:10]
#Only showing 10 otherwise will be very long

---